<a href="https://colab.research.google.com/github/yeats126/hg-da/blob/main/%EA%B3%B5%EA%B0%90%EC%9D%98_AI%EC%B1%97%EB%B4%87_%EB%A7%8C%EB%93%A4%EA%B8%B0_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 공감하는 말하기 챗봇 만들기

트랜스포머의 인코더 부분을 사용하여 한국어 문장 인코딩(벡터화)

In [ ]:
!pip install -q sentence-transformers

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

## SentenceBERT를 이용한 문장 인코딩

https://huggingface.co/jhgan/ko-sroberta-multitask

In [ ]:
encoder = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = encoder.encode(sentences)

print(embeddings)

## 데이터셋 로드

In [ ]:
from google.colab import drive

# 구글 드라이브를 마운트합니다.
drive.mount('/content/gdrive')

import shutil

# 구글 드라이브의 파일을 Colab 세션 내부로 복사합니다.
shutil.copy("/content/gdrive/My Drive/worries.csv", "/content/worries.csv")


df = pd.read_csv('/content/worries.csv')

df.head()

## 데이터 전처리

- 필요없는 칼럼 제거
- 챗봇 내용 없는 행 제거

In [ ]:
print(len(df))

df = df.dropna()

print(len(df))

## 챗봇의 원리

In [ ]:
df.loc[0, '유저']

In [ ]:
encoder.encode(df.loc[0, '유저'])

## 모든 데이터셋 인코딩

In [ ]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(encoder.encode(x)))

df.head()

## 예제 문장 입력

In [ ]:
text = '요즘 친구가 괴롭히네요.'

embedding = encoder.encode(text)

embedding

## 입력된 문장에 대해 유사도 계산

In [ ]:
df['similarity'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

## 유사도가 가장 높은 답변을 채택

In [ ]:
answer = df.loc[df['similarity'].idxmax()]

print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['similarity'])

## 심리상담 챗봇 앱

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr

def greet(user):
    embedding = encoder.encode(user)

    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['distance'].idxmax()]

    return answer['챗봇']

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f537fade1f68d52e7b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def greet(user, history=[]):
    embedding = encoder.encode(user)

    df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
    answer = df.loc[df['distance'].idxmax()]

    history.append([user, answer['챗봇']])

    return history, history

demo = gr.Interface(fn=greet, inputs=["text", "state"], outputs=["chatbot", "state"])

demo.launch(debug=True, share=True)